In [1]:
!apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
!pip install yt_dlp openai-whisper gradio torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=cadbf60c498a12a7ce1b973c428ce600a32bce73c9060159e0fa778c50248e70
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
import os, subprocess, whisper, gradio as gr

In [4]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 68.0MiB/s]


In [5]:
def format_timestamp(seconds: float) -> str:
    milliseconds = int((seconds % 1) * 1000)
    seconds = int(seconds)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

In [6]:
def write_srt(result, filename="subtitles.srt"):
    with open(filename, "w", encoding="utf-8") as f:
        for i, seg in enumerate(result["segments"], start=1):
            start = format_timestamp(seg["start"])
            end = format_timestamp(seg["end"])
            text = seg["text"].strip()
            f.write(f"{i}\n{start} --> {end}\n{text}\n\n")
    return filename

In [7]:
def generate_subtitles(youtube_url):
    try:
        # Download YouTube audio with yt_dlp (reliable)
        audio_file = "audio.mp3"
        !yt-dlp -x --audio-format mp3 -o {audio_file} {youtube_url}

        # Transcribe using Whisper
        result = model.transcribe(audio_file)
        srt_path = write_srt(result, "subtitles.srt")

        return result["text"], srt_path

    except Exception as e:
        return f"⚠️ Error: {str(e)}", None

In [8]:
gr.Interface(
    fn=generate_subtitles,
    inputs=gr.Textbox(label="Enter YouTube URL"),
    outputs=[gr.Textbox(label="Transcript"), gr.File(label="Download Subtitles (.srt)")],
    title=" YouTube Subtitle Generator ",
    description="Paste a YouTube video URL "
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75a7ad80d77baf2839.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
